<a href="https://colab.research.google.com/github/rubyvanrooyen/observation_planning/blob/main/comet67P/67P_orbit_from_XEphem_ephemeris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Orbital plots for comet 67P/Churyumov-Gerasimenko

## Installations
Use `skyfield` python package and published data files downloaded from the Minor Planet Center.    
https://rhodesmill.org/skyfield/kepler-orbits.html

In [ ]:
!pip install skyfield

Skyfield loads orbital elements from text files using the Pandas library.

In [2]:
!pip install pandas

MeerKAT observations uses `katpoint` objects and the `PyEphem` library

In [3]:
!pip install katpoint

     |████████████████████████████████| 102kB 8.8MB/s 


## Skyfield comet definition

Build a dataframe of comets

In [9]:
from skyfield.api import load
from skyfield.data import mpc

with load.open(mpc.COMET_URL) as f:
    comets = mpc.load_comets_dataframe(f)

print(len(comets), 'comets loaded')
print(mpc.COMET_URL)

899 comets loaded
https://www.minorplanetcenter.net/iau/MPCORB/CometEls.txt


In [10]:
# Keep only the most recent orbit for each comet,
# and index by designation for fast lookup.
comets = (comets.sort_values('reference')
          .groupby('designation', as_index=False).last()
          .set_index('designation', drop=False))

In [11]:
comets

,designation,perihelion_year,perihelion_month,perihelion_day,perihelion_distance_au,eccentricity,argument_of_perihelion_degrees,longitude_of_ascending_node_degrees,inclination_degrees,magnitude_g,magnitude_k,reference
designation,,,,,,,,,,,,
100P/Hartley,100P/Hartley,2022,8,10.9830,2.017170,0.412314,181.9888,37.6981,25.5648,9.0,4.0,MPEC 2013-O31
101P/Chernykh,101P/Chernykh,2020,1,12.6329,2.344628,0.595986,277.7044,116.2287,5.0527,10.0,4.8,MPEC 2020-FB5
102P/Shoemaker,102P/Shoemaker,2021,1,22.4082,2.069990,0.457095,20.5473,339.3658,25.8849,6.5,8.0,MPC 86232
103P/Hartley,103P/Hartley,2023,10,12.8107,1.066176,0.693281,181.2826,219.7326,13.6020,8.5,8.0,MPEC 2018-HA5
104P/Kowal,104P/Kowal,2022,1,11.1267,1.072861,0.665479,227.1260,207.3847,5.7200,16.0,4.0,MPC100283
...,...,...,...,...,...,...,...,...,...,...,...,...
P/2021 J3 (ATLAS),P/2021 J3 (ATLAS),2019,7,4.5182,4.904301,0.450806,125.7272,111.2261,14.5602,7.0,4.0,MPEC 2021-N06
P/2021 L2 (Leonard),P/2021 L2 (Leonard),2021,7,24.5739,1.938525,0.521741,51.5496,266.6779,21.0700,17.0,4.0,MPEC 2021-M74
P/2021 L4 (PANSTARRS),P/2021 L4 (PANSTARRS),2019,9,5.3092,2.787228,0.119425,234.7506,243.2235,16.9701,13.5,4.0,MPC xxxxx


In [12]:
skyfield_comet67P = comets.loc['67P/Churyumov-Gerasimenko']
print(skyfield_comet67P)

designation                            67P/Churyumov-Gerasimenko
perihelion_year                                             2021
perihelion_month                                              11
perihelion_day                                            2.0542
perihelion_distance_au                                   1.21064
eccentricity                                            0.649796
argument_of_perihelion_degrees                           22.1261
longitude_of_ascending_node_degrees                      36.3341
inclination_degrees                                       3.8713
magnitude_g                                                   11
magnitude_k                                                    4
reference                                          MPEC 2021-N06
Name: 67P/Churyumov-Gerasimenko, dtype: object


## XEphem comet definition

In [14]:
a = skyfield_comet67P.perihelion_distance_au
P = a**(3./2.)
motion = 1./P

epoch=f'{skyfield_comet67P.perihelion_month}/{skyfield_comet67P.perihelion_day}/{skyfield_comet67P.perihelion_year}'
comet67P_def = f'{skyfield_comet67P.name},e,{skyfield_comet67P.inclination_degrees},{skyfield_comet67P.longitude_of_ascending_node_degrees},{skyfield_comet67P.argument_of_perihelion_degrees},{skyfield_comet67P.perihelion_distance_au},{motion},{skyfield_comet67P.eccentricity},0.0000,{epoch},2000,g {skyfield_comet67P.magnitude_g}, {skyfield_comet67P.magnitude_k}'
print(comet67P_def)

67P/Churyumov-Gerasimenko,e,3.8713,36.3341,22.1261,1.210644,0.7507153897030153,0.649796,0.0000,11/2.0542/2021,2000,g 11.0, 4.0


Ephemeris from Minor Planet & Comet Ephemeris Service for object `67P/Churyumov-Gerasimenko`    
https://www.minorplanetcenter.net/iau/MPEph/MPEph.html


In [15]:
# From MPEC 2021-N06
xephem_comet67P = '67P/Churyumov-Gerasimenko,e,3.8713,36.3348,22.1246,3.457120,0.1533319,0.649811,341.5921,07/05.0/2021,2000,g 11.0,4.0'

```
# From MPEC 2021-N06  67P/Churyumov-Gerasimenko,e,3.8713,36.3348,22.1246,3.457120,0.1533319,0.649811,341.5921,07/05.0/2021,2000,g 11.0,4.0
```

```
a = comet67P.perihelion_distance_au
P = a**(3./2.)
motion = 1./P (a = 3.457120; P = a**(3./2.); motion = 1./P = 0.15557088350169382)
epoch=f'{comet67P.perihelion_month}/{comet67P.perihelion_day}/{comet67P.perihelion_year}

67P/Churyumov-Gerasimenko,{comet67P.name}
e,
3.8712,{comet67P.inclination_degrees}
36.3361,{comet67P.longitude_of_ascending_node_degrees}
22.122,{comet67P.argument_of_perihelion_degrees}
1.21064,{comet67P.perihelion_distance_au}
0.7507191102938259,{motion}
0.649832,{comet67P.eccentricity}
0.0000,
11/2.0436/2021,{epoch}
2000,
g 11.0,{comet67P.magnitude_g}
4.0{comet67P.magnitude_k}
```


## Orbital element
Creating a `katpoint` recognisable target as an `xephem` body (`katpoint.Target?` for docstring help)

```
The *xephem* body contains a string in XEphem EDB database format as the final field, with commas replaced by tildes.
If the name list is empty, the target name is taken from the XEphem string instead.
```

In [39]:
ts = load.timescale()
eph = load('de421.bsp')
sun, earth = eph['sun'], eph['earth']

from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN
comet = sun + mpc.comet_orbit(skyfield_comet67P, ts, GM_SUN)

from skyfield.api import S, E, wgs84
meerkat = earth + wgs84.latlon(30.7130 * S, 21.4430 * E)

t = ts.utc(2021, 7, 25)
#print(earth.at(t).observe(comet).radec())
#print(earth.at(t).observe(comet).apparent().radec())
#print(meerkat.at(t).observe(comet).radec())
#print(meerkat.at(t).observe(comet).apparent().radec())
print(meerkat.at(t).observe(comet).apparent().radec('date'))

(<Angle 01h 21m 32.72s>, <Angle +03deg 47' 03.8">, <Distance 1.16813 au>)


In [40]:
import ephem
yh = ephem.readdb(xephem_comet67P)
print(yh)
yh.compute('2021/7/25')
print(yh.name)
print("%s %s" % (yh.ra, yh.dec))
print("%s %s" % (ephem.constellation(yh), yh.mag))

<ephem.EllipticalBody '67P/Churyumov-Gerasimenko' at 0x7f14d5752030>
67P/Churyumov-Gerasimenko
1:21:32.36 3:47:00.0
('Psc', 'Pisces') 13.64


In [57]:
import katpoint
import numpy
from datetime import datetime

kat = katpoint.Antenna("ref, -30:42:39.8, 21:26:38.0, 1035.0, 0.0, , , 1.15")
observer = kat.observer
observer.horizon = numpy.deg2rad(20)
datetime_ = datetime(year=2021, month=7, day=25, hour=00, minute=00, second=00)
observer.date = ephem.date(datetime_)
print(observer)

target_def = xephem_comet67P.replace(',', '~')
target_str = f"67P/Churyumov-Gerasimenko, xephem, {target_def}"
katpoint_comet67P = katpoint.Target(target_str)
katpoint_comet67P.body.compute(observer)
print(katpoint_comet67P.body.name, katpoint_comet67P.body.mag, katpoint_comet67P.body.ra, katpoint_comet67P.body.dec)

<ephem.Observer date='2021/7/25 00:00:00' epoch='2000/1/1 12:00:00' lon='21:26:38.0' lat='-30:42:39.8' elevation=1035.0m horizon=20:00:00.0 temp=15.0C pressure=0.0mBar>
67P/Churyumov-Gerasimenko 13.64 1:21:32.72 3:47:04.1


## Output

In [60]:
print(skyfield_comet67P)
print(xephem_comet67P)

print('Ephem')
print(yh.name)
print("%s %s" % (yh.ra, yh.dec))

print('Skyfield')
print(meerkat.at(t).observe(comet).radec('date'))

print('katpoint')
print(katpoint_comet67P.body.name, katpoint_comet67P.body.mag, katpoint_comet67P.body.ra, katpoint_comet67P.body.dec)

designation                            67P/Churyumov-Gerasimenko
perihelion_year                                             2021
perihelion_month                                              11
perihelion_day                                            2.0542
perihelion_distance_au                                   1.21064
eccentricity                                            0.649796
argument_of_perihelion_degrees                           22.1261
longitude_of_ascending_node_degrees                      36.3341
inclination_degrees                                       3.8713
magnitude_g                                                   11
magnitude_k                                                    4
reference                                          MPEC 2021-N06
Name: 67P/Churyumov-Gerasimenko, dtype: object
67P/Churyumov-Gerasimenko,e,3.8713,36.3348,22.1246,3.457120,0.1533319,0.649811,341.5921,07/05.0/2021,2000,g 11.0,4.0
Ephem
67P/Churyumov-Gerasimenko
1:21:32.36 3:47:00.0
Sky

# Notes

Definition of XEphem targets URL: http://www.clearskyinstitute.com/xephem/help/xephem.html#mozTocId468501

7.1.2.1 General format rules    
Each object occupies one line in the file.    
The order of objects in a file does not matter.    
Lines beginning with anything other than a-z, A-Z or 0-9 are ignored and may be used for comments.    
Lines are separated into Fields using commas (,).    
Fields may be further subdivided into Subfields with vertical bars (|).    
All date fields may be in either of two forms:    
month/day/year, where day may contain a fractional portion. examples: 1/1/1993 and 1/1.234/1993 . Note the format of dates in database files is always M/D/Y, regardless of the current XEphem Date format Preference setting; or    
the year as real number as indicated by the presence of a decimal point, such as 1993.123.

```
C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0
```

* Field 1: Name
* Field 2: Type, e = heliocentric elliptical orbit. If Field 2 is e the object type is elliptical heliocentric (eccentricity < 1) and the remaining fields are defined as follows
* Field 3: i = inclination, degrees
* Field 4: O = longitude of ascending node, degrees
* Field 5: o = argument of perihelion, degrees
* Field 6: a = mean distance (aka semi-major axis), AU
* Field 7: n = mean daily motion, degrees per day (computed from a**3/2 if omitted)
* Field 8: e = eccentricity, must be < 1
* Field 9: M = mean anomaly, i.e., degrees from perihelion
* Field 10: E = epoch date, i.e., time of M
  * SubField 10A	First date these elements are valid, optional
  * SubField 10B	Last date these elements are valid, optional
* Field 11: D = the equinox year, i.e., time of i, O and o
* Field 12: First component of magnitude model, either g from (g,k) or H from (H,G). Specify which by preceding the number with a "g" or an "H". In absence of either specifier the default is (H,G) model. See Magnitude models.
* Field 13: Second component of magnitude model, either k or G
* Field 14: s = angular size at 1 AU, arc seconds, optional


Example provided for `katpoint` using C/2002 Y1 (Juels-Holvorcem)


When you load minor objects like comets and asteroids, the resulting object specifies the orbital elements that allow XEphem to predict its position.   
If you lack a catalog from which to load an object, you can start by creating a raw body of one of the following types and filling in its elements.    
Element attribute names start with underscores to distinguish them from the normal Body attributes that are set as the result of calling compute().
```
EllipticalBody elements:

_inc — Inclination (°)
_Om — Longitude of ascending node (°)
_om — Argument of perihelion (°)
_a — Mean distance from sun (AU)
_M — Mean anomaly from the perihelion (°)
_epoch_M — Date for measurement _M
_size — Angular size (arcseconds at 1 AU)
_e — Eccentricity
_epoch — Epoch for _inc, _Om, and _om
_H, _G — Parameters for the H/G magnitude model
_g, _k — Parameters for the g/k magnitude model
```

In [ ]:
import ephem
yh = ephem.readdb("C/2002 Y1 (Juels-Holvorcem),e,103.7816," +
                  "166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000," +
                  "04/13.2508/2003,2000,g  6.5,4.0")
yh.compute('2003/4/11')
print(yh.name)
print("%s %s" % (yh.ra, yh.dec))
print("%s %s" % (ephem.constellation(yh), yh.mag))
print(yh.writedb())

print(yh._inc, yh._Om)

C/2002 Y1 (Juels-Holvorcem)
0:22:44.58 26:49:48.1
('And', 'Andromeda') 5.96
C/2002 Y1 (Juels-Hol,e,103.7816,166.2194,128.8232,242.5695,0,0.9970576,0, 4/13.2508/2003, 1/01/2000,g6.5,4,0
103:46:53.8 166:13:09.9


```
C/2002 Y1 (Juels-Holvorcem), e,    103.7816,    166.2194,  128.8232,   242.5695,  0.0002609, 0.99705756,   0.0000,  04/13.2508/2003, 2000,    g  6.5,          4.0    
name                       , type, inclination, longitude, perihelion, distance,  motion,    eccentricity, anomaly, epoch,           equinox, First magnitude, Second magnitude
```

Information from    
IAU database URL: https://www.minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id=C%2F2002+Y1    
and URL: http://www.aerith.net/comet/catalog/2002Y1/2002Y1.html


You may have other parameters available for elliptical orbits that can be converted into these. The following relationships might be useful:
```
P = sqrt(a*a*a)
p = O + o
n = 0.9856076686/P
T = E - M/n
q = a*(1-e)
AU = 149,597,870 km = 92,955,621 U.S. statute miles
```
where
```
P = the orbital period, years;
p = longitude of perihelion, degrees
T = epoch of perihelion (add multiples of P for desired range)
q = perihelion distance, AU
```
Note that if you know T you can then set E = T and M = 0.

In [ ]:
# distance and motion calculations
a = 0.7138080 / (1. - 0.9971648)
P = a**(3./2.)
print(f'distance = {a}, motion = {1./P}')

distance = 251.76636568848423, motion = 0.0002503245440563847


# Elevation plots